In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
from mediapipe.framework.formats import landmark_pb2
from statistics import mean as mn
import mediapipe as mp
import cv2 as cv 
import numpy as np




In [2]:
cap = cv2.VideoCapture(0)



In [3]:

ptime= 0
mpdraw = mp.solutions.drawing_utils
mpfacemesh = mp.solutions.face_mesh
selfie_segmentation = mp.solutions.selfie_segmentation
facemesh = mpfacemesh.FaceMesh(max_num_faces=1)
drawspec = mpdraw.DrawingSpec(thickness=1,circle_radius=0,color= (224,224,224))
drawspec2 = mpdraw.DrawingSpec(thickness=1,circle_radius=0,color= (0,224,224))


In [ ]:

lk = [(159,145)]
rk = [(158,153)]


ale = [(33,246),(246,161),(161,160),(160,159),(159,158),(158,157),(157,173),(173,133),(133,155),(155,154),(154,153),(153,145),(145,144),(144,163),(163,7),(7,33)]



LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

# Left eye indices list
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
# Right eye indices list
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]

mp_face_mesh = mp.solutions.face_mesh


with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
) as face_mesh:

    while True:

        width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT )
        pa = np.full((int(height), int(width), 3), (0,0,0), dtype=np.uint8)

        suc, img = cap.read()



        segmented_image = img.copy( )
        tightness = .3


        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = facemesh.process(imgRGB)
        '''results2 = selfie_segmentation.process(img)'''
        '''image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)'''
        '''condition = np.stack((results.segmentation_mask,)*3 , axis=-1) > 0.1'''

        if results.multi_face_landmarks:
            for facelms in results.multi_face_landmarks:

                mpdraw.draw_landmarks(pa, facelms,lk,drawspec,drawspec)
                mpdraw.draw_landmarks(pa, facelms,rk,drawspec,drawspec)
                #mpdraw.draw_landmarks(imgRGB, facelms,lk)
                mpdraw.draw_landmarks(imgRGB, facelms,ale,drawspec2,drawspec2)


                xco=[]
                yco=[]
                for id,lms in enumerate(facelms.landmark):
                    ih,iw,ic = img.shape
                    x,y = int(lms.x*iw),int(lms.y*ih)
                    #print(id,x,y)
                    xco.append(x)
                    yco.append(y)
                    
                
                cv2.line(img, (xco[159],yco[159]),(xco[144],yco[144]), (0,255,255),3)
                cv2.line(img, (xco[158],yco[158]),(xco[153],yco[153]), (0,255,255),3)

                ml = (yco[159]-yco[144])/(xco[159]-xco[144])
                mr = (yco[158]-yco[153])/(xco[158]-xco[153])
                
                bl = yco[159]-(ml*xco[159])
                br = yco[153]-(mr*xco[153])
                
                print(bl,br)
                
                
                
                
                s = 1
                ls_single_face=results.multi_face_landmarks[0].landmark
                for idx in ls_single_face:
                    #print(idx.x,idx.y,idx.z)
                    s =s+1






        ctime = time.time()

        fps = 1/(ctime-ptime)
        ptime = ctime
        cv2.putText(img, f'FPS:{int(fps)}',(10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
        

        ret,frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h,img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            #print(results.multi_face_landmarks[0].landmark)
            mesh_points = np.array([np.multiply([p.x,p.y],[img_w,img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            #print(mesh_points[RIGHT_IRIS])
            #cv.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)


            sdx=mesh_points[RIGHT_IRIS][1][0]
            swx=mesh_points[RIGHT_IRIS][3][0]
            dwx = int(np.mean([sdx,swx]))

            sdy=mesh_points[RIGHT_IRIS][0][1]
            swy=mesh_points[RIGHT_IRIS][2][1]
            dwy = int(np.mean([sdy,swy]))

            ssdx = mesh_points[RIGHT_IRIS][0][0]
            sswx = mesh_points[RIGHT_IRIS][2][0]
            ddwx = int(np.mean([ssdx,sswx]))

            ssdy = mesh_points[RIGHT_IRIS][1][1]
            sswy = mesh_points[RIGHT_IRIS][3][1]
            ddwx = int(np.mean([ssdy,sswy]))
            
            

            '''cv2.line(frame, (ssdx,sdy),(sswx,swy), (0,0,255),3)
            cv2.line(frame, (sdx,ssdy),(swx,sswy), (0,0,255),3)'''
            #print(yco[159])
            #cv2.line(frame, (xco[159],y[159]),(x[149],y[149]), (0,0,255),3)
            cv2.line(img,(dwx,ddwx),(dwx,ddwx),(255,255,0),3)
            cv2.line(frame,(dwx,ddwx),(dwx,ddwx),(255,255,0),3)

            if ml> 0:
                if ddwx > (ml*dwx + bl):
                    cv2.putText(img, 'RIGHT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                else:
                    
                    if mr> 0:
                        if ddwx > (mr*dwx + br):
                            cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                        else:
                            cv2.putText(img, 'LEFT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                    elif mr<0:
                        if ddwx < (mr*dwx +br):
                            cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                        else:
                            cv2.putText(img, 'LEFT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)

                    
                    
                    
                    
            elif ml<0:
                if ddwx < (ml*dwx +bl):
                    cv2.putText(img, 'RIGHT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                else:
                    if mr> 0:
                        if ddwx > (mr*dwx + br):
                            cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                        else:
                            cv2.putText(img, 'LEFT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                    elif mr<0:
                        if ddwx < (mr*dwx +br):
                            cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                        else:
                            cv2.putText(img, 'LEFT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)

                   
            
                    
            '''if mr> 0:
                if ddwx > (mr*dwx + br):
                    cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                else:
                    cv2.putText(img, 'RIGHT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
            elif mr<0:
                if ddwx < (mr*dwx +br):
                    cv2.putText(img, '',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
                else:
                    cv2.putText(img, 'RIGHT',(10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 1)
            '''

            
            
        cv2.imshow('imgg',img)
        cv2.imshow('image',pa)
        cv2.imshow('RGB',imgRGB)
        cv2.imshow('hj',segmented_image)
        cv2.imshow('img',frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
#print(s)
